In [19]:
import glob
from PIL import Image
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import time

from bs4 import BeautifulSoup

In [108]:

#display all columns (not just first and last 5)
pd.options.display.max_columns = None

In [ ]:
# for traffic limit:
# https://www.sports-reference.com/bot-traffic.html

In [10]:
user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0',
]

In [3]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    #'if-modified-since': 'Thu, 22 May 2025 17:41:23 GMT',
    'priority': 'u=0, i',
#     'sec-ch-ua': '"Chromium";v="136", "Google Chrome";v="136", "Not.A/Brand";v="99"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    # 'cookie': 'osano_consentmanager_uuid=ef80ee15-7ba1-4791-b7f2-865f9fe1c04c; _ga=GA1.1.1181001228.1745513799; _lc2_fpi=6fbc725567fc--01jsmbnqy8vjns34qjv841trve; _lc2_fpi_meta=%7B%22w%22%3A1745513799624%7D; _lr_env_src_ats=false; gamera_user_id=5af48dbf-cb2e-47b9-b2c4-1589b5c9046a; _cc_id=8c2dd72220b52c18ac50d5761d80a8f8; _sharedid=f1f0c558-d27f-460c-acba-6a279f2d6dd5; ccuid=7af0fbcb-c7c8-4e3b-a7dd-65f5557092f6; hubspotutk=a37a24785af3ff877ac47ba5d442e0a1; sr_lang_views-en=-0.5; sr_note_box_countdown=0; srcssfull=yes; is_live=true; _li_dcdm_c=.fbref.com; _sharedid_cst=9SzrLEgsPQ%3D%3D; hb_insticator_uid=2d837a14-b8e1-458e-a45b-ac46df6472b3; __qca=P1-23bef6ae-42c3-49f0-8f40-c41262b84fe5; pbjs_fabrickId=%7B%22fabrickId%22%3A%22E1%3AA1Rn-osazp14LkUI92JoX3tozWvc9GiPcd-uEuIQjTKgdq9K8xaD_Ja3wlh4NGMdyUJbfnc69T4wWDXPT7-NuF0LK1a9Vur-IJWK_vLJh48%22%7D; panoramaId_expiry=1748541760781; panoramaId=f0c938dd0a501317554515c9903a4945a70239d951ede15abcb9c97f2bdc1af9; panoramaIdType=panoIndiv; _lr_sampling_rate=100; __hssrc=1; sr_n=2%7CThu%2C%2022%20May%202025%2018%3A03%3A24%20GMT; __hstc=218152582.a37a24785af3ff877ac47ba5d442e0a1.1745513801734.1747936973589.1747943711872.4; _lr_retry_request=true; __cf_bm=GHv6JqUTD4Srt_9i9ZjZa2w4KJgnEv0JSj0GIhpOWPM-1747947424-1.0.1.1-_BH3H8Y35BtFtanAdUxCHAiNzZz6BQfvydtNIJwd23R1NcIxTT5mpc9sEFQn.3vJOY9Xf7Sgjn8H.QtsxtliFVWShpj.2pqfc1HtvWA.U4A; _ga_80FRT7VJ60=GS2.1.s1747943712$o5$g1$t1747947737$j0$l0$h0; _ga_T897NZ0GWZ=GS2.1.s1747943712$o5$g1$t1747947737$j0$l0$h0; cto_bidid=iFeFpV9VYUloYkF4YlZORVpYOHhoaVMwcjNBJTJCeHFwdlo1eERYaG1NNnlOSFglMkZVQnMyUGdHbGhCaHBEcDF5SSUyRlhzdjdCRnowRjQwOXlFNXJuNzZ2ciUyRjlzOUMyNFJKMzh1bXFiZUdPdzZDQmNaWm1DVVVHR1NYRng4aHVHbDNGMUliV3hIM29URU1ENnglMkJCVnJKSUpFJTJGZFRPMUElM0QlM0Q; __hssc=218152582.38.1747943711872; cto_bundle=jdqoEF9JeVB4dUVFNGIxdXRzJTJGd3JwdTElMkZkQTRNMHhUV05WUGdneEpYT3Q1Q0ZSV0ttbyUyQkRLQnRxb3pDU3V4dmlINyUyRnoxYTNRMiUyQk5qc2FIRFJwMU9ZdzF4NCUyQkxGbXdsbnlhNXhzejRBQ3BuUjdOMG1oYmk0NzlwNk4yR2VjR0JnVjRjMkFjNGVuc2NxM2duZCUyRkclMkJzV3Rjd0FYemtFb0VIRVZQclpqT0dwZGVxWGhhZUViQXdFVWYwaGEzb2Z1JTJCSVNJem9zVHowbWVITGJ6Zkp3RENKVzNUc1RNT1pyTDhRYTJuOWh0R1gzb0lNbzRZM0p6a1FpMGFsSUpXd2tCY1klMkIyUG1MN0I0WGVUQm5QejFBelU1cnhCNkoyTjZ6dG9xNllaM2JlQjZ0NUNsNlZiZ24xdjkxTTJDbEhvZHJ3anRFeENV; __gads=ID=1c580f67fe1263ae:T=1745513801:RT=1747947739:S=ALNI_MaBE8QPVcTxwnzu0LRWr89WWJoXiQ; __gpi=UID=0000109e519e38cb:T=1745513801:RT=1747947739:S=ALNI_MZY0HNGr161e1YwanVGVlUWbvL4Fg; __eoi=ID=5a6200b8b106bde6:T=1745513801:RT=1747947739:S=AA-AfjbtTQ054d4DRuJSOarDextj; pbjs_fabrickId_cst=riwMLDMsbg%3D%3D',
}

In [4]:
fpl_dir = '/Users/yuta/Desktop/fpl_web_scrape/2024-2025/FB Reference'

# Step 1: Get all EPL Teams

In [6]:
#start with premier league page
#get hrefs to all teams page to scrape player name and ids
url_epl_teams = 'https://fbref.com/en/comps/9/Premier-League-Stats'
response = requests.get(url_epl_teams, headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

epl_teams_table = soup.find("table", id="results2024-202591_overall")
epl_teams_href = epl_teams_table.find_all('td', attrs = {'data-stat': 'team'})

team_hrefs_list = []
for i in epl_teams_href:
    x = i.find('a', href=True)
    team_hrefs_list.append(x['href'])
    
team_hrefs_list = list(set(team_hrefs_list))


In [7]:
#make sure there are 20 distinct teams
assert len(team_hrefs_list) == 20

# Step 2: Get all Player ID's + Season Cumulative Stats

In [8]:
#cumulative df for all players
season_cumulative_df = pd.DataFrame()

for short_url in team_hrefs_list:
    team_url = f"https://fbref.com{short_url}"
    print(team_url)

    #scrape all player hrefs and append to list
    response = requests.get(team_url)
    soup = BeautifulSoup(response.content, "html.parser")

    players_table = soup.find("table", id="stats_standard_9")
    team_df = pd.read_html(str(players_table))[0]
    team_df.columns = [col[1] for col in team_df.columns]

    team_df['team'] = short_url.split("/")[-1].replace('-Stats', '').replace('-', ' ')


    #add href column
    players_href = players_table.find_all('th', attrs = {'data-stat': 'player', 'scope':'row'})
    players_href_list = []
    for i in players_href:
        try:
            x = i.find('a', href=True)
            players_href_list.append(x['href'])
        except:
            print(i)

    players_href_list.append('') #add one for squad total row
    players_href_list.append('') #add one for squad total row

    team_df['player_link'] = players_href_list
            
    season_cumulative_df = season_cumulative_df.append(team_df, ignore_index=True)
    time.sleep(5)
   
#export to csv
season_cumulative_df.to_csv(f"{fpl_dir}/fb_ref_epl_2024_2025_cumulative_stats.csv", index=False)

https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
<th class="left" data-stat="player" scope="row">Squad Total</th>
<th class="left" data-stat="player" scope="row">Opponent Total</th>
https://fbref.com/en/squads/33c895d4/Southampton-Stats
<th class="left" data-stat="player" scope="row">Squad Total</th>
<th class="left" data-stat="player" scope="row">Opponent Total</th>
https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
<th class="left" data-stat="player" scope="row">Squad Total</th>
<th class="left" data-stat="player" scope="row">Opponent Total</th>
https://fbref.com/en/squads/19538871/Manchester-United-Stats
<th class="left" data-stat="player" scope="row">Squad Total</th>
<th class="left" data-stat="player" scope="row">Opponent Total</th>
https://fbref.com/en/squads/fd962109/Fulham-Stats
<th class="left" data-stat="player" scope="row">Squad Total</th>
<th class="left" data-stat="player" scope="row">Opponent Total</th>
https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
<th class="l

In [9]:
season_cumulative_df.head(40)

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches,team,player_link
0,Moisés Caicedo,ec ECU,"MF,DF",22.0,38,38,3351.0,37.2,1.0,2.0,...,0.03,0.08,0.02,0.08,0.10,0.02,0.10,Matches,Chelsea,/en/players/16264a81/Moises-Caicedo
1,Cole Palmer,eng ENG,"MF,FW",22.0,37,36,3191.0,35.5,15.0,8.0,...,0.31,0.54,0.49,0.31,0.79,0.38,0.68,Matches,Chelsea,/en/players/dc7f8a28/Cole-Palmer
2,Levi Colwill,eng ENG,DF,21.0,35,35,3149.0,35.0,2.0,1.0,...,0.06,0.09,0.05,0.03,0.09,0.05,0.09,Matches,Chelsea,/en/players/700783e7/Levi-Colwill
3,Marc Cucurella,es ESP,DF,26.0,36,33,2988.0,33.2,5.0,1.0,...,0.15,0.18,0.08,0.06,0.15,0.08,0.15,Matches,Chelsea,/en/players/1daec722/Marc-Cucurella
4,Enzo Fernández,ar ARG,MF,23.0,36,32,2947.0,32.7,6.0,7.0,...,0.18,0.40,0.19,0.21,0.39,0.19,0.39,Matches,Chelsea,/en/players/5ff4ab71/Enzo-Fernandez
5,Robert Sánchez,es ESP,GK,26.0,32,32,2880.0,32.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Chelsea,/en/players/6a713852/Robert-Sanchez
6,Nicolas Jackson,sn SEN,FW,23.0,30,28,2220.0,24.7,10.0,5.0,...,0.41,0.61,0.50,0.17,0.67,0.50,0.67,Matches,Chelsea,/en/players/9c36ed83/Nicolas-Jackson
7,Noni Madueke,eng ENG,FW,22.0,32,27,2033.0,22.6,7.0,3.0,...,0.31,0.44,0.43,0.19,0.61,0.43,0.61,Matches,Chelsea,/en/players/bf34eebd/Noni-Madueke
8,Pedro Neto,pt POR,FW,24.0,35,24,2269.0,25.2,4.0,6.0,...,0.16,0.40,0.14,0.27,0.40,0.14,0.40,Matches,Chelsea,/en/players/7ba2eaa9/Pedro-Neto
9,Malo Gusto,fr FRA,DF,21.0,32,19,1862.0,20.7,0.0,1.0,...,0.00,0.05,0.08,0.07,0.15,0.08,0.15,Matches,Chelsea,/en/players/d56b9520/Malo-Gusto


# Step 3: Get Per Match Stats by Player

In [20]:
season_cumulative_df= pd.read_csv(f"{fpl_dir}/fb_ref_epl_2024_2025_cumulative_stats.csv")

season_cumulative_df.head()

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,team,player_link
0,Moisés Caicedo,ec ECU,"MF,DF",22.0,38,38,3351.0,37.2,1.0,2.0,...,0.03,0.08,0.02,0.08,0.10,0.02,0.10,Matches,Chelsea,/en/players/16264a81/Moises-Caicedo
1,Cole Palmer,eng ENG,"MF,FW",22.0,37,36,3191.0,35.5,15.0,8.0,...,0.31,0.54,0.49,0.31,0.79,0.38,0.68,Matches,Chelsea,/en/players/dc7f8a28/Cole-Palmer
2,Levi Colwill,eng ENG,DF,21.0,35,35,3149.0,35.0,2.0,1.0,...,0.06,0.09,0.05,0.03,0.09,0.05,0.09,Matches,Chelsea,/en/players/700783e7/Levi-Colwill
3,Marc Cucurella,es ESP,DF,26.0,36,33,2988.0,33.2,5.0,1.0,...,0.15,0.18,0.08,0.06,0.15,0.08,0.15,Matches,Chelsea,/en/players/1daec722/Marc-Cucurella
4,Enzo Fernández,ar ARG,MF,23.0,36,32,2947.0,32.7,6.0,7.0,...,0.18,0.40,0.19,0.21,0.39,0.19,0.39,Matches,Chelsea,/en/players/5ff4ab71/Enzo-Fernandez


In [124]:
def df_from_url(url):
    df = pd.read_html(url)[0]

    df.columns = [' '.join(col).strip() for col in df.columns]

    df = df.reset_index(drop=True)
    df.head()

    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
          new_col = col.split()[-1] 
        else:
          new_col = col
        new_columns.append(new_col)

    df.columns = new_columns
    
    #drop nan rows
    df = df[df['Date'].notnull()]

    return df

In [123]:
player_urls = (
    season_cumulative_df
    .query('Min>0')
    .query('player_link != ""')
    .query('Player != "Squad Total"')
    .query('Player != "Opponent Total"')
    [['team', 'player_link']]
    .drop_duplicates()
    .to_dict(orient='index')
)


In [70]:
team_list = list(
    season_cumulative_df
    [['team']]
    .drop_duplicates()
    .team
)
team_list

['Chelsea',
 'Southampton',
 'Bournemouth',
 'Manchester United',
 'Fulham',
 'Arsenal',
 'Wolverhampton Wanderers',
 'Tottenham Hotspur',
 'West Ham United',
 'Manchester City',
 'Ipswich Town',
 'Nottingham Forest',
 'Aston Villa',
 'Brentford',
 'Newcastle United',
 'Liverpool',
 'Everton',
 'Brighton and Hove Albion',
 'Leicester City',
 'Crystal Palace']

In [ ]:
#go team by team

In [85]:
player_list[0].split(sep="/")[-1]

'Dean-Henderson'

In [ ]:
team_list = ['West Ham United',
 'Manchester City',
 'Ipswich Town',
 'Nottingham Forest',
 'Aston Villa',
 'Brentford',]

In [ ]:
for team in team_list:
    print(team)
    player_weekly_stats_by_team = pd.DataFrame()
    
    player_list = (
        season_cumulative_df
        .query(f'team=="{team}"')
        .query('Min>0')
        .query('player_link != ""')
        .query('Player != "Squad Total"')
        .query('Player != "Opponent Total"')
        [['player_link']]
        .drop_duplicates()
        .reset_index()
        .player_link
    )
    
    print(len(player_list))
    
    for player in player_list:
        player_name = player.split(sep="/")[-1]
        player_id = player.split(sep="/")[-2]
        
        print(f"   {player_name}")
        
        df_summary = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/summary/{player_name}-Match-Logs')
        time.sleep(6)
        df_passing = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/passing/{player_name}-Match-Logs')
        time.sleep(6)
        df_pass_types = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/passing_types/{player_name}-Match-Logs')
        time.sleep(6)
        df_shot_goal = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/gca/{player_name}-Match-Logs')
        time.sleep(6)
        df_posession = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/possession/{player_name}-Match-Logs')
        time.sleep(6)
        df_defensive_action = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/defense/{player_name}-Match-Logs')
        time.sleep(6)
        
        df_all = (
            df_summary
            .merge(df_passing.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
            .merge(df_pass_types.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
            .merge(df_shot_goal.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
            .merge(df_posession.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
            .merge(df_defensive_action.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
        )
        
        df_all['team'] = team
        df_all['player_name'] = player_name
        df_all['player_id'] = player_id
        
        player_weekly_stats_by_team = pd.concat([player_weekly_stats_by_team, df_all], ignore_index=True)
        time.sleep(3)
        
    player_weekly_stats_by_team.to_csv(f"{fpl_dir}/{team} - Weekly Player Stats.csv", index=False)
        

In [96]:
player = player_list[6]

player_name = player.split(sep="/")[-1]
player_id = player.split(sep="/")[-2]

In [97]:
player_name

'Eberechi-Eze'

In [128]:
df_summary = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/summary/{player_name}-Match-Logs')
time.sleep(6)
df_passing = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/passing/{player_name}-Match-Logs')
time.sleep(6)
df_pass_types = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/passing_types/{player_name}-Match-Logs')
time.sleep(6)
df_shot_goal = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/gca/{player_name}-Match-Logs')
time.sleep(6)
df_posession = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/possession/{player_name}-Match-Logs')
time.sleep(6)
df_defensive_action = df_from_url(f'https://fbref.com/en/players/{player_id}/matchlogs/2024-2025/c9/defense/{player_name}-Match-Logs')
time.sleep(6)

In [129]:
df_all = (
    df_summary
    .merge(df_passing.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
    .merge(df_pass_types.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
    .merge(df_shot_goal.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
    .merge(df_posession.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
    .merge(df_defensive_action.drop('Report', axis=1), on=['Date', 'Day', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min'], how='outer')
)

In [130]:
len(df_all)

34

In [131]:
df_all

,Date,Day,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Performance Gls,Performance Ast,Performance PK,Performance PKatt,Performance Sh,Performance SoT,Performance CrdY,Performance CrdR,Performance Touches,Performance Tkl,Performance Int,Performance Blocks,Expected xG,Expected npxG,Expected xAG,SCA SCA,SCA GCA,Passes Cmp,Passes Att,Passes Cmp%,Passes PrgP,Carries Carries_x,Carries PrgC_x,Take-Ons Att_x,Take-Ons Succ_x,Report,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,Short Cmp,Short Att,Short Cmp%,Medium Cmp,Medium Att,Medium Cmp%,Long Cmp,Long Att,Long Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Att,Pass Types Live,Pass Types Dead,Pass Types FK,Pass Types TB,Pass Types Sw,Pass Types Crs,Pass Types TI,Pass Types CK,Corner Kicks In,Corner Kicks Out,Corner Kicks Str,Outcomes Cmp,Outcomes Off,Outcomes Blocks,SCA Types SCA,SCA Types PassLive,SCA Types PassDead,SCA Types TO,SCA Types Sh,SCA Types Fld,SCA Types Def,GCA Types GCA,GCA Types PassLive,GCA Types PassDead,GCA Types TO,GCA Types Sh,GCA Types Fld,GCA Types Def,Touches Touches,Touches Def Pen,Touches Def 3rd,Touches Mid 3rd,Touches Att 3rd,Touches Att Pen,Touches Live,Take-Ons Att_y,Take-Ons Succ_y,Take-Ons Succ%,Take-Ons Tkld,Take-Ons Tkld%,Carries Carries_y,Carries TotDist,Carries PrgDist,Carries PrgC_y,Carries 1/3,Carries CPA,Carries Mis,Carries Dis,Receiving Rec,Receiving PrgR,Tackles Tkl,Tackles TklW,Tackles Def 3rd,Tackles Mid 3rd,Tackles Att 3rd,Challenges Tkl,Challenges Att,Challenges Tkl%,Challenges Lost,Blocks Blocks,Blocks Sh,Blocks Pass,Int,Tkl+Int,Clr,Err
0,2024-08-18,Sun,Matchweek 1,Away,L 1–2,Crystal Palace,Brentford,Y,"AM,LM",90.0,0.0,0.0,0.0,0.0,7.0,3.0,0.0,0.0,58.0,1.0,0.0,0.0,0.4,0.4,0.0,6.0,0.0,31.0,43.0,72.1,5.0,41.0,7.0,6.0,2.0,Match Report,31.0,43.0,72.1,417.0,101.0,21.0,24.0,87.5,10.0,13.0,76.9,0.0,1.0,0.0,0.0,0.0,0.1,0.0,5.0,2.0,0.0,5.0,43.0,38.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31.0,2.0,0.0,6.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0,0.0,1.0,30.0,27.0,5.0,58.0,6.0,2.0,33.3,4.0,66.7,41.0,260.0,177.0,7.0,4.0,2.0,2.0,1.0,43.0,7.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2024-08-24,Sat,Matchweek 2,Home,L 0–2,Crystal Palace,West Ham,Y,AM,90.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,64.0,2.0,0.0,1.0,0.4,0.4,0.3,6.0,0.0,36.0,44.0,81.8,5.0,41.0,6.0,7.0,2.0,Match Report,36.0,44.0,81.8,667.0,151.0,13.0,15.0,86.7,20.0,22.0,90.9,2.0,5.0,40.0,0.0,0.3,0.2,4.0,3.0,0.0,0.0,5.0,44.0,35.0,8.0,3.0,0.0,0.0,5.0,0.0,2.0,2.0,0.0,0.0,36.0,1.0,1.0,6.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,7.0,27.0,31.0,6.0,64.0,7.0,2.0,28.6,5.0,71.4,41.0,189.0,102.0,6.0,3.0,4.0,2.0,2.0,49.0,10.0,2.0,2.0,0.0,2.0,0.0,1.0,2.0,50.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0
2,2024-09-01,Sun,Matchweek 3,Away,D 1–1,Crystal Palace,Chelsea,Y,AM,90.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,51.0,1.0,0.0,1.0,0.2,0.2,0.0,2.0,0.0,33.0,39.0,84.6,9.0,37.0,1.0,5.0,2.0,Match Report,33.0,39.0,84.6,627.0,238.0,14.0,15.0,93.3,15.0,17.0,88.2,4.0,4.0,100.0,0.0,0.0,0.1,1.0,3.0,2.0,0.0,9.0,39.0,33.0,5.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,33.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,0.0,7.0,32.0,14.0,3.0,51.0,5.0,2.0,40.0,3.0,60.0,37.0,162.0,86.0,1.0,0.0,1.0,3.0,1.0,39.0,7.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,50.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,2024-09-14,Sat,Matchweek 4,Home,D 2–2,Crystal Palace,Leicester City,Y,"AM,LW",90.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,73.0,2.0,0.0,2.0,0.5,0.5,0.2,10.0,1.0,54.0,61.0,88.5,12.0,51.0,3.0,2.0,1.0,Match Report,54.0,61.0,88.5,915.0,268.0,26.0,28.0,92.9,19.0,21.0,90.5,6.0,8.0,75.0,0.0,0.2,0.3,3.0,4.0,5.0,0.0,12.0,61.0,51.0,10.0,2.0,1.0,1.0,3.0,1.0,5.0,3.0,0.0,0.0,54.0,0.0,2.0,10.0,7.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0.0,5.0,30.0,39.0,5.0,73.0,2.0,1.0,50.0,1.0,50.0,51.0,289.0,142.0,3.0,3.0,1.0,0.0,2.0,54.0,14.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,0.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0
4,2024-09-21,Sat,Matchweek 5,Home,D 0–0,Crystal Palace,Manchester Utd,Y,AM,89.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.

In [ ]:
#urls

#summary
#https://fbref.com/en/players/e342ad68/matchlogs/2024-2025/c9/Mohamed-Salah-Match-Logs
#<a href="/en/players/e342ad68/matchlogs/2024-2025/c9/summary/Mohamed-Salah-Match-Logs">Summary</a>

#passing 
#https://fbref.com/en/players/e342ad68/matchlogs/2024-2025/c9/passing/Mohamed-Salah-Match-Logs

#passing types
#<a href="/en/players/e342ad68/matchlogs/2024-2025/c9/passing_types/Mohamed-Salah-Match-Logs">Pass Types</a>

#shot and goal creation
#<a href="/en/players/e342ad68/matchlogs/2024-2025/c9/gca/Mohamed-Salah-Match-Logs">Goal and Shot Creation</a>

#possession
#<a href="/en/players/e342ad68/matchlogs/2024-2025/c9/possession/Mohamed-Salah-Match-Logs">Possession</a>

#defensive action
#<a href="/en/players/e342ad68/matchlogs/2024-2025/c9/defense/Mohamed-Salah-Match-Logs">Defensive Actions</a>